### Scrapping des données :

In [1]:
from selenium import webdriver
import pandas as pd

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [38]:

browser = webdriver.Firefox()
browser.get('https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/172961152W?motsCles=data%20&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesConvention=143687&secteursActivite=101753&page=27&selectedIndex=3')
#browser.get('https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/172856347W?motsCles=data%20&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesConvention=143687&page=58&selectedIndex=0')



###Le lien à scrapper pour que les métier DATA
#https://www.apec.fr/candidat/recherche-emploi.html/emploi?motsCles=data%20&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesConvention=143687&secteursActivite=101753&page=0

# Tests 

In [31]:
conteneur_des_resultats = browser.find_element(By.CLASS_NAME,"container-result")# recuprer la div qui contient l'ensemble d'annonces
resultats = conteneur_des_resultats.find_elements(By.XPATH,"./div/a") # Recuperer les liens vers le descriptif de chaque annonce
resultats.pop()
for element in resultats:
    print(element.get_attribute('href'))

https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/172982802W?motsCles=data%20&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesConvention=143687&secteursActivite=101753&page=0&selectedIndex=0
https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/173012464W?motsCles=data%20&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesConvention=143687&secteursActivite=101753&page=0&selectedIndex=1
https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/173104275W?motsCles=data%20&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesConvention=143687&secteursActivite=101753&page=0&selectedIndex=2
https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/173082168W?motsCles=data%20&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesConvention=143687&secteursActivite=101753&page=0&selectedIndex=3
https://www.apec.fr/candidat/recherche-emploi.ht

# Passage à la page de description de la première annonce

In [32]:
script = 'window.location.href = "'+resultats[0].get_attribute('href')+'";'
browser.execute_script(script)

# Création d'un dictionnaire pour stocker les information de chaque annonces
On récupére  : 'l'entreprise','type_contrat','deparetement','date_publication','date_mise_a_jour','salaire','prise_de_poste','experience','metier','statut_du_poste','zone_de_deplacement','secteur_d_activite','Teletravail','descriptif_poste','profil_recherche','langues','savoir_etre','savoir_faire' et'descriptif_entreprise'


In [39]:
informations_cles = ['entreprise','type_contrat','deparetement','date_publication','date_mise_a_jour','salaire','prise_de_poste','experience','metier','statut_du_poste','zone_de_deplacement','secteur_d_activite','Teletravail','descriptif_poste','profil_recherche','langues','savoir_etre','savoir_faire','descriptif_entreprise']

# Initialiser un dictionnaire avec des clés et des valeurs initiales à une liste vide
dictionnaire_annonces ={}
for cle in informations_cles:
    dictionnaire_annonces[cle]=[]
dictionnaire_annonces

{'entreprise': [],
 'type_contrat': [],
 'deparetement': [],
 'date_publication': [],
 'date_mise_a_jour': [],
 'salaire': [],
 'prise_de_poste': [],
 'experience': [],
 'metier': [],
 'statut_du_poste': [],
 'zone_de_deplacement': [],
 'secteur_d_activite': [],
 'Teletravail': [],
 'descriptif_poste': [],
 'profil_recherche': [],
 'langues': [],
 'savoir_etre': [],
 'savoir_faire': [],
 'descriptif_entreprise': []}

In [204]:
dictionnaire_annonces

{'entreprise': [],
 'type_contrat': [],
 'deparetement': [],
 'date_publication': [],
 'date_mise_a_jour': [],
 'salaire': [],
 'prise_de_poste': [],
 'experience': [],
 'metier': [],
 'statut_du_poste': [],
 'zone_de_deplacement': [],
 'secteur_d_activite': [],
 'Teletravail': [],
 'descriptif_poste': [],
 'profil_recherche': [],
 'langues': [],
 'savoir_etre': [],
 'savoir_faire': [],
 'descriptif_entreprise': []}

In [40]:
informations_cles = ['entreprise','type_contrat','deparetement','date_publication','date_mise_a_jour','salaire','prise_de_poste','experience','metier','statut_du_poste','zone_de_deplacement','secteur_d_activite','Teletravail','descriptif_poste','profil_recherche','langues','savoir_etre','savoir_faire','descriptif_entreprise']

# Initialiser un dictionnaire avec des clés et des valeurs initiales à une liste vide
dictionnaire_annonces ={}
for cle in informations_cles:
    dictionnaire_annonces[cle]=[]

def click_bouton_Vor_Plus(browser) :
    #cliquer sur voir plus 
    button = browser.find_element(By.XPATH, "//span[text()='Voir plus']")
    button.click()
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Voir plus']"))).click()

def recuperer_une_offre(browser , dictionnaire_annonces):

    # entreprise , type_contrat , departement
    entete_annonce = browser.find_element(By.CLASS_NAME,"card-offer__text")# recuprer la div du petit réctangle
    informations = entete_annonce.find_elements(By.XPATH,"./ul/li") # Recuperer les liens vers le descriptif de chaque annonce
    for i,info in enumerate(informations):
        cle = informations_cles[i]
        mot = info.text.strip()
        dictionnaire_annonces[cle].append(mot)



    # 'date_publication','date_mise_a_jour'
    date_publication = entete_annonce.find_elements(By.CLASS_NAME,"date-offre")[0].text
    date_mise_a_jour = entete_annonce.find_elements(By.CLASS_NAME,"date-offre")[1].text
    dictionnaire_annonces['date_publication'].append(date_publication)
    dictionnaire_annonces['date_mise_a_jour'].append(date_mise_a_jour)


    # 'salaire','prise_de_poste','experience','metier','statut_du_poste','zone_de_deplacement','secteur_d_activite','Teletravail'
    apec_poste_imformations = browser.find_element(By.TAG_NAME,"apec-poste-informations")# recuprer la div du petit réctangle
    liste_informations = apec_poste_imformations.find_elements(By.XPATH,"//div[contains(@class, 'details-post')]")
    salaire = None
    prise_de_poste= None
    experience = None
    metier = None
    statut_du_poste = None
    zone_de_deplacement = None
    secteur_d_activite = None
    Teletravail = None
    for information in liste_informations:
        reponse = information.find_elements(By.XPATH, "./h4[text()='Salaire']/following-sibling::span")
        if reponse :
            salaire = reponse[0].text
            
        reponse = information.find_elements(By.XPATH, "./h4[text()='Prise de poste']/following-sibling::span")
        if reponse :
            prise_de_poste = reponse[0].text
            
        reponse = information.find_elements(By.XPATH, "./h4[text()='Expérience']/following-sibling::span")
        if reponse :
            experience = reponse[0].text
            
        reponse = information.find_elements(By.XPATH, "./h4[text()='Métier']/following-sibling::span")
        if reponse :
            metier = reponse[0].text

        reponse = information.find_elements(By.XPATH, "./h4[text()='Statut du poste']/following-sibling::span")
        if reponse :
            statut_du_poste = reponse[0].text
            
        reponse = information.find_elements(By.XPATH, "./h4[text()='Zone de déplacement']/following-sibling::span")
        if reponse :
            zone_de_deplacement = reponse[0].text

        reponse = information.find_elements(By.XPATH, "./h4[text()='Secteur d’activité du poste']/following-sibling::span")
        if reponse :
            secteur_d_activite = reponse[0].text     
        
        reponse = information.find_elements(By.XPATH, "./h4[text()='Télétravail']/following-sibling::span")
        if reponse :
            Teletravail = reponse[0].text    

    dictionnaire_annonces['salaire'].append(salaire)
    dictionnaire_annonces['prise_de_poste'].append(prise_de_poste)
    dictionnaire_annonces['experience'].append(experience)
    dictionnaire_annonces['metier'].append( metier)
    dictionnaire_annonces['statut_du_poste'].append(statut_du_poste)
    dictionnaire_annonces['zone_de_deplacement'].append(zone_de_deplacement)
    dictionnaire_annonces['secteur_d_activite'].append(secteur_d_activite)
    dictionnaire_annonces['Teletravail'].append(Teletravail)


    #'descriptif_poste' 'profil_recherche' 
    descriptifs = apec_poste_imformations.find_elements(By.XPATH,"//div[contains(@class, 'details-post')]")
    descriptif_poste = None
    descriptif_profil = None
    for desc in descriptifs:
        reponse = desc.find_elements(By.XPATH, "./h4[text()='Descriptif du poste']/following-sibling::p")
        if reponse:
            descriptif_poste=reponse[0].text
        reponse = desc.find_elements(By.XPATH, "./h4[text()='Profil recherché']/following-sibling::p")
        if reponse:
            descriptif_profil=reponse[0].text

    dictionnaire_annonces['descriptif_poste'].append(descriptif_poste)
    dictionnaire_annonces['profil_recherche'].append(descriptif_profil)

    # Langues
    langues_requises = apec_poste_imformations.find_element(By.CLASS_NAME,"added-skills-manager__language")
    liste_langues_requises= langues_requises.find_elements(By.XPATH, "//h5[text()='Langues']/../following-sibling::*")
    Langues_L = []
    for langue in liste_langues_requises:
        text_lu = langue.text
        if text_lu != "Voir moins" :
            Langues_L.append(langue.text)
    dictionnaire_annonces['langues'].append(Langues_L)


    #savoir-être
    soft_skills = apec_poste_imformations.find_element(By.CLASS_NAME,"added-skills-manager__knowhow")
    liste_soft_skills = soft_skills.find_elements(By.XPATH, "//h5[text()='Savoir-être']/../following-sibling::*")

    soft_skills_L = []
    for sf in liste_soft_skills:
        text_lu = sf.text
        if text_lu != "Voir moins" :
            soft_skills_L.append(sf.text)
    dictionnaire_annonces['savoir_etre'].append(soft_skills_L)

    #savoir-faire
    hard_skills = apec_poste_imformations.find_element(By.CLASS_NAME,"added-skills-manager__knowledge")
    liste_hard_skills = hard_skills.find_elements(By.XPATH, "//h5[text()='Savoir-faire']/../following-sibling::*")

    hard_skills_L = []
    for hs in liste_hard_skills:
        #print(hs.get_attribute("innerHTML"))
        text_lu = hs.text
        if text_lu != "Voir moins" :
            hard_skills_L.append(hs.text)
    dictionnaire_annonces['savoir_faire'].append(hard_skills_L)

    #Discption d'entreprise
    descriptif_entreprise = None
    desc_entreprise_element = apec_poste_imformations.find_element(By.XPATH, "//h4[text()='Entreprise']/following-sibling::p")
    if desc_entreprise_element:
            descriptif_entreprise=desc_entreprise_element.text
    dictionnaire_annonces["descriptif_entreprise"].append(descriptif_entreprise)

    
    

def offre_suivante(browser):
    next_page_button = browser.find_element(By.CLASS_NAME,"nextpage")
    next_page_button.click()

def check_dict_list_lengths(d):
    # Récupère les longueurs de toutes les listes
    lengths = [len(v) for v in d.values()]

    # Vérifie si toutes les longueurs sont identiques
    return all(length == lengths[0] for length in lengths)


NB_JOBS = 6264
click_bouton_Vor_Plus(browser)
for i in range(NB_JOBS):
    try:
        print(i)
        recuperer_une_offre(browser ,dictionnaire_annonces)
        if not check_dict_list_lengths(dictionnaire_annonces) :
            print("erreur taille")
            break
        
        
        element = WebDriverWait(browser,10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME,"nextpage"))
        )
        element[0].click()
        
    except:
        print("Pas d'offre suivantes")
        break
        #browser.quit()

#https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/172900450W?motsCles=data%20&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesConvention=143687&selectedIndex=0&page=20
#dictionnaire_annonces

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [41]:
#suppression à partir de 4015 + 1516
for cle, liste_dictionnaires in dictionnaire_annonces.items():
    taille_liste = len(liste_dictionnaires)  # Calcul de la taille de la liste
    print(f"La taille de la liste de la clé '{cle}' est : {taille_liste}")

La taille de la liste de la clé 'entreprise' est : 1430
La taille de la liste de la clé 'type_contrat' est : 1430
La taille de la liste de la clé 'deparetement' est : 1430
La taille de la liste de la clé 'date_publication' est : 1430
La taille de la liste de la clé 'date_mise_a_jour' est : 1430
La taille de la liste de la clé 'salaire' est : 1430
La taille de la liste de la clé 'prise_de_poste' est : 1430
La taille de la liste de la clé 'experience' est : 1430
La taille de la liste de la clé 'metier' est : 1430
La taille de la liste de la clé 'statut_du_poste' est : 1430
La taille de la liste de la clé 'zone_de_deplacement' est : 1430
La taille de la liste de la clé 'secteur_d_activite' est : 1430
La taille de la liste de la clé 'Teletravail' est : 1430
La taille de la liste de la clé 'descriptif_poste' est : 1430
La taille de la liste de la clé 'profil_recherche' est : 1430
La taille de la liste de la clé 'langues' est : 1430
La taille de la liste de la clé 'savoir_etre' est : 1430
La

In [26]:
for cle, liste_dictionnaires in dictionnaire_annonces.items():
        #if cle=='entreprise' or cle == 'type_contrat' or cle == 'deparetement' or cle=='date_publication' or cle=='date_mise_a_jour' :#
        #if cle=="salaire" or cle=='prise_de_poste' or cle=='experience' or cle=='metier' or cle=='statut_du_poste' or cle=='zone_de_deplacement' or cle=='secteur_d_activite' or cle=='Teletravail' or cle=='descriptif_poste' or cle=='profil_recherche' or cle=='langues' or cle=='savoir_etre':
        if cle == "savoir_faire" or cle=="descriptif_entreprise"  :    
                liste_dictionnaires.pop()
        

## Création du dataFrame

In [42]:
import pandas as pd
df = pd.DataFrame(dictionnaire_annonces)
df.tail(20)

,entreprise,type_contrat,deparetement,date_publication,date_mise_a_jour,salaire,prise_de_poste,experience,metier,statut_du_poste,zone_de_deplacement,secteur_d_activite,Teletravail,descriptif_poste,profil_recherche,langues,savoir_etre,savoir_faire,descriptif_entreprise
1410,Vialink,1 \n CDI,Puteaux - 92,Publiée le 26/01/2024,Actualisée le 26/01/2024,A négocier,Dès que possible,Minimum 10 ans,Key account manager,Cadre du secteur privé,Pas de déplacement,EDITION DE LOGICIELS APPLICATIFS,Partiel possible,Descriptif du poste Nous recherchons un(e) Sen...,Profil recherché Votre profil Vous ne vous rec...,[Anglais],"[Autonomie, Bienveillance, Convivialité, Curio...","[Administratif, CRM, Négociation commerciale, ...",Qui sommes-nous ? La mission de VIALINK : simp...
1411,Mobiapps,1 \n CDI,Neuilly-sur-Seine - 92,Publiée le 18/01/2024,Actualisée le 25/01/2024,A négocier,Dès que possible,Minimum 3 ans,Architecte réseaux,Cadre du secteur privé,Régionale,CONSEIL EN SYSTÈMES ET LOGICIELS INFORMATIQUES,None,Vos missions :\nEn tant qu'Expert Modern Manag...,"Vous êtes le candidat que nous recherchons, si...",[Aucune langue attendue],[Aucun savoir-être attendu],"[Amélioration continue, Architecture SI, Autom...","Mobiapps fête ses 10 ans, continuons à écrire ..."
1412,PLATEFORME DES DONNEES DE SANTE,1 \n CDI,Paris 15 - 75,Publiée le 15/02/2024,Actualisée le 15/02/2024,A négocier,Dès que possible,Minimum 7 ans,Chef de projet digital,Cadre du secteur privé,Pas de déplacement,"TRAITEMENT DE DONNÉES, HÉBERGEMENT ET ACTIVITÉ...",Partiel possible,"Sous la supervision du directeur technique, vo...",Compétences techniques\nUne expérience de mini...,"[Anglais, Français]","[Autonomie, Capacité d'adaptation, Clarté, Cur...","[Analyse des besoins, Appui technique, Bureaut...",Comment améliorer les dépistage et faire en so...
1413,PLATEFORME DES DONNEES DE SANTE,1 \n CDI,Paris 15 - 75,Publiée le 15/02/2024,Actualisée le 15/02/2024,A négocier,Dès que possible,Tous niveaux d'expérience acceptés,Directeur marketing,Cadre du secteur privé,Pas de déplacement,"TRAITEMENT DE DONNÉES, HÉBERGEMENT ET ACTIVITÉ...",Partiel possible,La Direction scientifique est chargée d’élabor...,Nous recherchons un profil diplômé d’Ecole de ...,[Anglais],"[Autonomie, Capacité d'adaptation, Clarté, Esp...","[Cartographie, Administratif, Benchmarking, Bu...",Comment améliorer les dépistage et faire en so...
1414,Apsia,1 \n CDI,Paris 09 - 75,Publiée le 13/02/2024,Actualisée le 13/02/2024,A négocier,14/02/2024,Minimum 2 ans,Ingénieur de maintenance informatique,Cadre du secteur privé,Internationale,CONSEIL EN SYSTÈMES ET LOGICIELS INFORMATIQUES,Partiel possible,"Profitant d’une forte croissance, nous sommes ...",Tu as/es...\nDiplômé(e) d’une école d’ingénieu...,[Anglais],"[Autonomie, Curiosité, Esprit d'équipe, Implic...","[Comité de pilotage, Communication, Développem...",Qui sommes-nous ?\nNotre métier ? Transformer ...
1415,PLATEFORME DES DONNEES DE SANTE,1 \n CDI,Paris 15 - 75,Publiée le 09/02/2024,Actualisée le 09/02/2024,A négocier,Dès que possible,Tous niveaux d'expérience acceptés,Contrôleur de gestion,Cadre du secteur privé,Pas de déplacement,"TRAITEMENT DE DONNÉES, HÉBERGEMENT ET ACTIVITÉ...",Partiel possible,"Sous la supervision du directeur technique, vo...",Compétences techniques indispensables \nConnai...,[Aucune langue attendue],"[Autonomie, Capacité d'analyse, Capacité d’éco...","[Achats, Budget prévisionnel, Cloud, Contrôle ...",Comment améliorer les dépistage et faire en so...
1416,Dogfinance,1 \n CDI,Nantes - 44,Publiée le 31/01/2024,Actualisée le 31/01/2024,A négocier,Dès que possible,Tous niveaux d'expérience acceptés,Ingénieur sûreté de fonctionnement,Cadre du secteur privé,Pas de déplacement,PROGRAMMATION INFORMATIQUE,None,"BCEF IT, l'informatique de la Banque Commercia...",Êtes-vous notre prochain Ingénieur Risques It ...,"[Anglais, Français]","[Capacité d'adaptation, Capacité d'analyse, Ca...","[Analyse des risques, Documentation, Action de...",Pourquoi rejoin

## Création du fichier csv

In [43]:
df.to_csv(f'./DATA_INFO/Apec_data_2.csv', index=False)  # Assurez-vous de remplacer 'nom_du_fichier.csv' par le nom de fichier souhaité

# Fermeture de la fenetre ouverte 

In [2]:
browser.quit()